In [1]:
pip install mpi4py

Note: you may need to restart the kernel to use updated packages.


In [7]:
#Ex 1:
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(comm.Get_rank())
if comm.Get_rank()==0:
    print("Numer of nodes", comm.Get_size())

0
Numer of nodes 1


In [15]:
#Ex 2:
from mpi4py import MPI 
import numpy

comm = MPI.COMM_WORLD 
rank = comm.Get_rank()

if rank == 0:
    idata = 43
    comm.send(idata, dest=0)
    
elif rank == 1:
    idata = comm.recv(source=0)
    print('On process 0, data is ',idata)

In [18]:
#Ex 3:

from mpi4py import MPI 
import numpy

comm = MPI.COMM_WORLD 
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14} 
    comm.send(data, dest=0)
    
elif rank == 1:
    data = comm.recv(source=0)
    print('On process 1, data is ',data)

In [22]:
#Ex 4: 
from mpi4py import MPI 
import numpy as np

comm = MPI.COMM_WORLD 
rank = comm.Get_rank()

if rank == 0:
    numData = 10
    comm.send(numData, dest=0)
    
    data = np.linspace(0.0,3.14,numData) 
    comm.Send(data, dest=0)
    
elif rank == 1:
    numData = comm.recv(source=0)
    print('Number of data to receive: ',numData)
    
    data = np.empty(numData, dtype='d') 
    comm.Recv(data, source=0) 
    
    print('data received: ',data)

In [23]:
#Ex 5:
from mpi4py import MPI 

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'key1' : [1,2, 3],
            'key2' : ( 'abc', 'xyz')} 
else:
    data = None
    
data = comm.bcast(data, root=0)
print('Rank: ',rank,', data: ' ,data)

Rank:  0 , data:  {'key1': [1, 2, 3], 'key2': ('abc', 'xyz')}


In [24]:
#Ex 6:
from mpi4py import MPI 
import numpy as np

comm = MPI.COMM_WORLD 
rank = comm.Get_rank()

if rank == 0:
    numData = 10
    data = np.linspace(0.0,3.14,numData)
else:
    numData = None
    
numData = comm.bcast(numData, root=0) 

if rank != 0:
    data = np.empty(numData, dtype='d')
    
comm.Bcast(data, root=0)

print('Rank: ',rank, ', data received: ',data)

Rank:  0 , data received:  [0.         0.34888889 0.69777778 1.04666667 1.39555556 1.74444444
 2.09333333 2.44222222 2.79111111 3.14      ]


In [25]:
#Ex 7:
from mpi4py import MPI 
import numpy as np
comm = MPI.COMM_WORLD

size = comm.Get_size() 
rank = comm.Get_rank()

numDataPerRank = 10 
data = None

if rank == 0:
    data = np.linspace(1,size*numDataPerRank,numDataPerRank*size) 
    print('Original array before scatting', data)
    
recvbuf = np.empty(numDataPerRank, dtype='d') 
comm.Scatter(data, recvbuf, root=0)

print('Rank: ',rank, ', recvbuf received: ',recvbuf)

Original array before scatting [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Rank:  0 , recvbuf received:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [26]:
#Ex 8: 
import numpy as np 
from mpi4py import MPI 
comm = MPI.COMM_WORLD

t_start = MPI.Wtime()

data = np.zeros(5)
for i in range(0, len(data), 1):
    data[i] = i
    
print('[%i]'%comm.rank, data)

if comm.rank==0:
    totals = np.zeros_like(data)
else:
    totals = None
    
comm.Reduce(
    [data, MPI.DOUBLE], 
    [totals, MPI.DOUBLE],
    op = MPI.SUM,
    root = 0
)
if comm.rank==0:
    print('[%i]'%comm.rank, totals)
    
comm.Barrier()
t_diff = MPI.Wtime() - t_start

if comm.rank==0:
    print(t_diff)

[0] [0. 1. 2. 3. 4.]
[0] [0. 1. 2. 3. 4.]
0.002605
